In [273]:
# Imports
import pandas as pd
import firebase_admin
from firebase_admin import credentials, db
import bcrypt

In [274]:
# Connect to FireBase DB
cred = credentials.Certificate('credentials.json')

if not firebase_admin._apps:
    cred = credentials.Certificate('credentials.json')
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://nature-notebook-db-default-rtdb.firebaseio.com/'
    })

In [275]:
# User class to store local data
class User:
    def __init__(self, user_id: int, username: str, email: str):
        self.user_id = user_id
        self.username = username
        self.email = email
        self.settings =  self.init_setting_map() # Dict for settings
        self.permissions = set()  # Set of granted permissions

    def init_setting_map(self):
        dict = {}
        dict["light_mode"] = True
        dict["user_active"] = True
        # TO DO: add more settings
        return dict

    def update_setting(self, key: str, value):
        self.settings[key] = value

    def get_setting(self, key: str):
        return self.settings.get(key, None)

    def add_permission(self, permission: str):
        self.permissions.add(permission)

    def remove_permission(self, permission: str):
        self.permissions.discard(permission)

    def check_permission(self, permission: str) -> bool:
        return permission in self.permissions

    def __repr__(self):
        return (f"User(id={self.user_id}, username='{self.username}')")

In [276]:
# Password hashing and checking functions
def hash_pass_text(password_text):
    # First time hashing password using bcrypt; the salt is saved into the hash itself; slow hash to prevent malicious brute force attacks
    password_bytes = password_text.encode('utf-8')
    return bcrypt.hashpw(password_bytes, bcrypt.gensalt(14))

def check_password(password_check, password_true):
    # Check hashed password
    password_check = password_check.encode('utf-8')
    password_true = password_true.encode('utf-8')
    return bcrypt.checkpw(password_check, password_true)


In [277]:
# Returns total user database
def get_users():
    ref = db.reference('users/')
    snapshot = ref.get()

    data = []    
    for username, user_info in snapshot.items():
        user_info['username'] = username  # Add username to the dictionary
        data.append(user_info)

    # Convert the list of dictionaries to a pandas DataFrame
    df = pd.DataFrame(data)

    columns = ['username'] + [col for col in df.columns if col != 'username'] # Rearrange columns to get username in front
    df = df[columns]

    return df # Return data

In [278]:
# Get all users from DB example
get_users()

,username,email,is_active,password_hash,species_found
0,carrot_and_pea_cookie,nkalode2@illinois.edu,True,$2b$14$ygN1P1ijoa6KmATx37rx4.a0hbdbL3mebK21yzm...,[-1]
1,chungus,billa@gmail.com,True,$2b$14$6C8CKYEzCgVXKhUdHdQTAOVtQzvujMj1BRKDtau...,[-1]
2,jethalal,jijaji@sundar.com,True,$2b$14$Bq.jG6UuQmBeUrnTj0zD3.42pvj4.EvIkqFmsrb...,[-1]
3,lagumists047,ks129@illinois.edu,True,$2b$14$xlr0N4XxXFEd2pzvmYnoreUfObcgC3jcr3W.bdW...,[-1]
4,linger1109,ishaang5@illinois.edu,True,$2b$14$P0nPrPSn74tC3VFfqpJ6teJ6ZBGN7KiFKQlbEIN...,[-1]
5,stevenpi12,stevenp8@illinois.edu,True,$2b$14$0k/kWhjxsvDutmPi5iB/FeCghJOEibCbteMFA7c...,[-1]


In [279]:
# Functions to add and remove user
def add_user(username, email, password):
    ref = db.reference(f"users/{username}")  
    if ref.get() is not None: # Check if user already exists
        return 
    
    # Create user dict to push
    user = {
        'email': email,
        'password_hash' : hash_pass_text(password).decode('utf-8'),
        'is_active': True,
        'species_found' : [-1] # Dummy value cause empty list wasn't been pushed to FireBase TODO: implement species found as 200 bit binary string
    }
    
    ref.set(user)

def remove_user(username):
    ref = db.reference(f"users/{username}")  
    ref.delete()

In [280]:
# Add and remove user example
add_user('jethalal', 'jijaji@sundar.com', 'JalebiMaster420')
add_user('chungus', 'bingus@yahoo.co.in', 'EvilLarry')
get_users()

,username,email,is_active,password_hash,species_found
0,carrot_and_pea_cookie,nkalode2@illinois.edu,True,$2b$14$ygN1P1ijoa6KmATx37rx4.a0hbdbL3mebK21yzm...,[-1]
1,chungus,billa@gmail.com,True,$2b$14$6C8CKYEzCgVXKhUdHdQTAOVtQzvujMj1BRKDtau...,[-1]
2,jethalal,jijaji@sundar.com,True,$2b$14$Bq.jG6UuQmBeUrnTj0zD3.42pvj4.EvIkqFmsrb...,[-1]
3,lagumists047,ks129@illinois.edu,True,$2b$14$xlr0N4XxXFEd2pzvmYnoreUfObcgC3jcr3W.bdW...,[-1]
4,linger1109,ishaang5@illinois.edu,True,$2b$14$P0nPrPSn74tC3VFfqpJ6teJ6ZBGN7KiFKQlbEIN...,[-1]
5,stevenpi12,stevenp8@illinois.edu,True,$2b$14$0k/kWhjxsvDutmPi5iB/FeCghJOEibCbteMFA7c...,[-1]


In [281]:
remove_user('jethalal')
get_users()

,username,email,is_active,password_hash,species_found
0,carrot_and_pea_cookie,nkalode2@illinois.edu,True,$2b$14$ygN1P1ijoa6KmATx37rx4.a0hbdbL3mebK21yzm...,[-1]
1,chungus,billa@gmail.com,True,$2b$14$6C8CKYEzCgVXKhUdHdQTAOVtQzvujMj1BRKDtau...,[-1]
2,lagumists047,ks129@illinois.edu,True,$2b$14$xlr0N4XxXFEd2pzvmYnoreUfObcgC3jcr3W.bdW...,[-1]
3,linger1109,ishaang5@illinois.edu,True,$2b$14$P0nPrPSn74tC3VFfqpJ6teJ6ZBGN7KiFKQlbEIN...,[-1]
4,stevenpi12,stevenp8@illinois.edu,True,$2b$14$0k/kWhjxsvDutmPi5iB/FeCghJOEibCbteMFA7c...,[-1]


In [282]:
# General getter function
def get_info(username, what):
    path_to_node = f'users/{username}/{what}'
    ref = db.reference(path_to_node)
    snapshot = ref.get()
    if snapshot is not None:    # If info exists, return it
        return snapshot

# Specific getters
def get_email(username):
    return get_info(username, 'email')

def get_species_found(username):
    return get_info(username, 'species_found')

def get_password(username):
    return get_info(username, 'password_hash')

def get_active_status(username):
    return get_info(username, 'is_active')


In [283]:
# Getters example
get_email('carrot_and_pea_cookie')

'nkalode2@illinois.edu'

In [284]:
# Function to validate password of an user 
def validate_password(username, pass_check):
    password_hash = get_password(username)
    return check_password(pass_check, password_hash)

In [285]:
# Validate password example
print(validate_password('chungus', 'EvilLarry'))
print(validate_password('chungus', 'EvilBarry'))

True
False


In [286]:
# Update observation helper function
def update_info(username, new_what, what): 
    ref = db.reference(f'users/{username}')  
    updates = { what : new_what } 
    ref.update(updates) # Push update

# Update observation's data functions
def update_username(username, new_username):
    update_info(username, new_username, 'user')

def update_email(username, new_email):
    update_info(username, new_email, 'email')

def update_password(username, new_pass):
    new_hash = hash_pass_text(new_pass).decode('utf-8')
    update_info(username, new_hash, 'password_hash')

def update_active(username):
    curr = get_active_status(username)
    update_info(username, not curr, 'is_active')

# Add a species id newly found by the user
def add_species_found(username, species_id):
    curr_found = get_species_found(username)
    if (species_id not in curr_found):  # Check if the bird was already found
        curr_found = curr_found + [species_id]
        update_info(username, curr_found, 'species_found')

# Reset the species found by the user
def remove_all_species_found(username):
    update_info(username, [-1], 'species_found')

In [287]:
# Update email, active example & remove species found example
update_email('chungus', 'billa@gmail.com')
update_active('stevenpi12')
remove_all_species_found('lagumists047')
get_users()

,username,email,is_active,password_hash,species_found
0,carrot_and_pea_cookie,nkalode2@illinois.edu,True,$2b$14$ygN1P1ijoa6KmATx37rx4.a0hbdbL3mebK21yzm...,[-1]
1,chungus,billa@gmail.com,True,$2b$14$6C8CKYEzCgVXKhUdHdQTAOVtQzvujMj1BRKDtau...,[-1]
2,lagumists047,ks129@illinois.edu,True,$2b$14$xlr0N4XxXFEd2pzvmYnoreUfObcgC3jcr3W.bdW...,[-1]
3,linger1109,ishaang5@illinois.edu,True,$2b$14$P0nPrPSn74tC3VFfqpJ6teJ6ZBGN7KiFKQlbEIN...,[-1]
4,stevenpi12,stevenp8@illinois.edu,False,$2b$14$0k/kWhjxsvDutmPi5iB/FeCghJOEibCbteMFA7c...,[-1]


In [288]:
# Kartikey found some birds (Add species example)
remove_user('chungus')
for i in range(1, 11):
    add_species_found('lagumists047', i)

get_users()

,username,email,is_active,password_hash,species_found
0,carrot_and_pea_cookie,nkalode2@illinois.edu,True,$2b$14$ygN1P1ijoa6KmATx37rx4.a0hbdbL3mebK21yzm...,[-1]
1,lagumists047,ks129@illinois.edu,True,$2b$14$xlr0N4XxXFEd2pzvmYnoreUfObcgC3jcr3W.bdW...,"[-1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]"
2,linger1109,ishaang5@illinois.edu,True,$2b$14$P0nPrPSn74tC3VFfqpJ6teJ6ZBGN7KiFKQlbEIN...,[-1]
3,stevenpi12,stevenp8@illinois.edu,False,$2b$14$0k/kWhjxsvDutmPi5iB/FeCghJOEibCbteMFA7c...,[-1]
